In [1]:
# model = Net()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
import torch

In [2]:
PATH = "checkpoint/plbart_token_level_embed_with_new_loss_batch_8_1657011226_epoch_0.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0503, requires_grad=True), tensor(0.0278, requires_grad=True))

In [3]:

PATH = "checkpoint/plbart_token_level_embed_with_new_loss_batch_8_1657011226_epoch_1.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0648, requires_grad=True), tensor(0.0278, requires_grad=True))

In [4]:

PATH = "checkpoint/plbart_token_level_embed_with_new_loss_batch_8_1657011226_epoch_2.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0646, requires_grad=True), tensor(0.0278, requires_grad=True))

In [6]:

PATH = "checkpoint/plbart_token_level_embed_with_new_loss_batch_8_1657011226_epoch_3.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0629, requires_grad=True), tensor(0.0277, requires_grad=True))

In [7]:

PATH = "checkpoint/plbart_token_level_embed_batch_8_1656726351_epoch_4.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0180, requires_grad=True), tensor(0.0263, requires_grad=True))

In [8]:

PATH = "checkpoint/plbart_token_level_embed_batch_8_1656726351_epoch_5.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0180, requires_grad=True), tensor(0.0263, requires_grad=True))

In [9]:

PATH = "checkpoint/plbart_token_level_embed_batch_8_1656726351_epoch_6.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0180, requires_grad=True), tensor(0.0263, requires_grad=True))

In [8]:

PATH = "checkpoint/plbart_token_level_embed_batch_8_1656726351_epoch_7.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0180, requires_grad=True), tensor(0.0263, requires_grad=True))

In [8]:

from data import (
    STYLE_DIM,
    get_data_loader,
    cluster_labels_no_outliers,
    train_dataset,
    test_dataset,
)

from model import InRepPlusGAN, Discriminator

generator = InRepPlusGAN(style_dim=STYLE_DIM)#.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
discriminator = Discriminator(
        vocab_size=generator.config.vocab_size,
        embedding_layer=generator.encoder.embed_tokens,
        embedding_dim=generator.config.d_model,
        output_size=128,
        style_dim=STYLE_DIM,
    )

In [9]:

from transformers import PLBartTokenizer
tokenizer = PLBartTokenizer.from_pretrained(
    "uclanlp/plbart-multi_task-python",
    language_codes="multi",
    src_lang="python",
    tgt_lang="python",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
input_batch = tokenizer(["def hello_world(): print('Hello World')"], max_length=1024, padding="max_length", return_tensors="pt")

In [6]:
input_batch["input_ids"].shape

torch.Size([1, 1024])

In [12]:
from utils.model_utils import label_tensor_to_one_hot

style_encoding = label_tensor_to_one_hot(
    torch.Tensor([1]).long(), STYLE_DIM
)#.to(device)

In [8]:
style_encoding

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:

# generator = InRepPlusGAN(style_dim=STYLE_DIM)#.to(device)
# generator.load_state_dict(checkpoint['g_state_dict'])
generator_output, modifier_output = generator(
    input_ids=input_batch["input_ids"],
    attention_mask=input_batch["attention_mask"],
    style_encoding=style_encoding,
)
generated_tokens = generator_output.logits.argmax(-1)
generated_text_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
generated_text_batch

In [14]:

generated_text_batch = tokenizer.batch_decode(generated_tokens)
generated_input_batch = tokenizer(generated_text_batch, max_length=1024, padding="max_length", return_tensors="pt")

In [17]:
generated_encoding = generator.get_encoding(**generated_input_batch)[0]

In [18]:
generated_encoding.shape

torch.Size([1, 1024, 768])

In [19]:
generated_encoding

tensor([[[-0.1696, -0.1361,  0.1512,  ...,  0.1206, -0.0296,  0.1654],
         [-0.2737,  0.4388,  0.0367,  ..., -0.3705,  0.0986,  0.9460],
         [-0.2157, -0.9253,  1.1519,  ..., -0.0858,  0.1850,  0.9281],
         ...,
         [ 1.0032, -1.0735, -0.1932,  ...,  1.2176,  1.2246,  0.0746],
         [ 0.8282, -1.1856, -0.1770,  ...,  1.2326,  1.4042,  0.1314],
         [ 0.8852, -1.1173, -0.2209,  ...,  1.2250,  1.3229,  0.1277]]])

In [14]:

generator = InRepPlusGAN(style_dim=STYLE_DIM)#.to(device)
generator.load_state_dict(checkpoint['g_state_dict'])
generator_output, modifier_output = generator(
    input_ids=input_batch["input_ids"],
    attention_mask=input_batch["attention_mask"],
    style_encoding=style_encoding,
)
generated_tokens = generator_output.logits.argmax(-1)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

NameError: name 'checkpoint' is not defined

In [15]:
batch_size, seq_len, hidden_dim = modifier_output.shape
modifier_output.shape

torch.Size([1, 1024, 768])

In [16]:
modifier_output.view(batch_size, seq_len*hidden_dim).shape

torch.Size([1, 786432])

In [21]:
cossim = torch.nn.CosineSimilarity(dim=-1, eps=1e-08)(generated_encoding, modifier_output)

In [24]:
-cossim.mean()

tensor(-0.0402, grad_fn=<NegBackward0>)

In [14]:

PATH = "checkpoint/plbart_token_level_embed_with_new_loss_batch_8_1657011226_epoch_0.pt"
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
checkpoint["g_loss"], checkpoint["d_loss"]

(tensor(0.0503, requires_grad=True), tensor(0.0278, requires_grad=True))

In [13]:

generator = InRepPlusGAN(style_dim=STYLE_DIM)#.to(device)
generator.load_state_dict(checkpoint['g_state_dict'])
generator_output, modifier_output = generator(
    input_ids=input_batch["input_ids"],
    attention_mask=input_batch["attention_mask"],
    style_encoding=style_encoding,
)
generated_tokens = generator_output.logits.argmax(-1)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return return retu

In [17]:
generator_output.logits.argmax(-1).shape

torch.Size([1, 1024])

In [1]:
from data import train_dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from datasets import Dataset

In [16]:
train_dataset = Dataset.from_dict(train_dataset[:64])

In [17]:
example = train_dataset["input_ids"][0]

In [19]:
tokenizer.batch_decode([example], skip_special_tokens=True)

['# coding: utf-8 """ Wavefront REST API <p>The Wavefront REST API enables you to interact with Wavefront servers using standard REST API tools. You can use the REST API to automate commonly executed operations such as automatically tagging sources.</p><p>When you make REST API calls outside the Wavefront REST API documentation you must add the header \\"Authorization: Bearer &lt;&lt;API-TOKEN&gt;&gt;\\" to your HTTP requests.</p> # noqa: E501 OpenAPI spec version: v2 Contact: chitimba@wavefront.com Generated by: https://github.com/swagger-api/swagger-codegen.git """ from __future__ import absolute_import import unittest import wavefront_api_client from wavefront_api_client.models.service_account import ServiceAccount # noqa: E501 from wavefront_api_client.rest import ApiException class TestServiceAccount(unittest.TestCase): """ServiceAccount unit test stubs""" def setUp(self): pass def tearDown(self): pass def testServiceAccount(self): """Test ServiceAccount""" # FIXME: construct obje

In [20]:
from utils.model_utils import label_tensor_to_one_hot

style_encoding = label_tensor_to_one_hot(
    torch.Tensor([1]).long(), STYLE_DIM
)#.to(device)

In [21]:
tokenizer.batch_decode([example], skip_special_tokens=True)

['# coding: utf-8 """ Wavefront REST API <p>The Wavefront REST API enables you to interact with Wavefront servers using standard REST API tools. You can use the REST API to automate commonly executed operations such as automatically tagging sources.</p><p>When you make REST API calls outside the Wavefront REST API documentation you must add the header \\"Authorization: Bearer &lt;&lt;API-TOKEN&gt;&gt;\\" to your HTTP requests.</p> # noqa: E501 OpenAPI spec version: v2 Contact: chitimba@wavefront.com Generated by: https://github.com/swagger-api/swagger-codegen.git """ from __future__ import absolute_import import unittest import wavefront_api_client from wavefront_api_client.models.service_account import ServiceAccount # noqa: E501 from wavefront_api_client.rest import ApiException class TestServiceAccount(unittest.TestCase): """ServiceAccount unit test stubs""" def setUp(self): pass def tearDown(self): pass def testServiceAccount(self): """Test ServiceAccount""" # FIXME: construct obje

In [28]:

style_encoding = label_tensor_to_one_hot(
    torch.Tensor([17]).long(), STYLE_DIM
)#.to(device)

generator = InRepPlusGAN(style_dim=STYLE_DIM)#.to(device)
generator.load_state_dict(checkpoint['g_state_dict'])
generator_output, modifier_output = generator(
    input_ids=torch.Tensor(example).long().unsqueeze(0),
    # attention_mask=input_batch["attention_mask"],
    style_encoding=style_encoding,
)
generated_tokens = generator_output.logits.argmax(-1)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['0 not 0 0 not not 0 not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not not 0 0 0 0 0 0 0 0 0 0 not 0 not 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 not 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 not not 0 not 0 0 0 not 0 not 0 not 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 not 0 0 0 0 0 0 not not not not not not not not not not not not not not not not not not no